In [6]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

# 1. 生成数据集
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)
features[:6], labels[:6]

(tensor([[ 0.0342,  0.8639],
         [-0.6793,  0.8812],
         [-1.6021,  1.6610],
         [ 0.3589, -0.8702],
         [-0.4901, -0.2682],
         [-0.4456,  0.3117]]),
 tensor([[ 1.3418],
         [-0.1599],
         [-4.6511],
         [ 7.8787],
         [ 4.1288],
         [ 2.2564]]))

In [7]:
# 2. 读取数据集
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [8]:
next(iter(data_iter))

[tensor([[ 1.1314, -0.5247],
         [-0.5167,  1.3623],
         [-1.0003, -0.2853],
         [-0.1742,  1.0970],
         [ 0.2268,  0.2210],
         [ 0.4041,  0.6887],
         [-0.0704, -0.1763],
         [-2.5771,  1.4997],
         [-0.3799, -1.5719],
         [ 0.4273, -1.3630]]),
 tensor([[ 8.2586],
         [-1.4702],
         [ 3.1589],
         [ 0.1245],
         [ 3.9165],
         [ 2.6740],
         [ 4.6565],
         [-6.0400],
         [ 8.7907],
         [ 9.6694]])]

In [9]:
# 3. 定义模型
## nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1)) # 2个输入特征，1个输出

In [10]:
net[0].weight.data.normal_(0, 0.01) # 初始化权重
net[0].bias.data.fill_(0) # 初始化偏差

tensor([0.])

In [12]:
loss = nn.MSELoss() # 均方误差损失函数

In [13]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03) # 随机梯度下降优化器

In [14]:
# 4. 训练模型
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000284
epoch 2, loss 0.000103
epoch 3, loss 0.000102


In [15]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0001, -0.0002])
b的估计误差： tensor([0.0008])
